# Travel Agent API - Quick Test Notebook
Use this notebook to smoke-test the FastAPI backend endpoints.
- Make sure the server is running (see next cell for a reminder).
- Requires: `requests` package.

## Start the API server (outside this notebook)
In a terminal (Windows cmd), activate the conda env, set the API key, and run the backend:

```cmd
conda activate gen-ai
set TRAVEL_AGENT_API_KEY=YOUR_SECRET_KEY
python -m backend.main
```

Default base URL: http://localhost:8000

Notes:
- Authenticated endpoint: POST /api/chat requires the API key via `Authorization: Bearer <key>` (used by this notebook) or `X-API-Key: <key>`.
- GET /health is open (no auth).

In [30]:
import os, json, time
import requests
from dotenv import load_dotenv
load_dotenv()
#BASE_URL = os.getenv('TRAVEL_AGENT_API_BASE', 'https://travel-mult-agent.azurewebsites.net')
BASE_URL = "http://localhost:8000"
API_KEY = os.getenv('TRAVEL_AGENT_API_KEY')  # required for authenticated endpoints

s = requests.Session()
if API_KEY:
    # Use Bearer scheme; backend also accepts X-API-Key
    s.headers.update({'Authorization': f'Bearer {API_KEY}'})

print('Using base URL:', BASE_URL)
print('Auth header set:', 'YES' if API_KEY else 'NO (set TRAVEL_AGENT_API_KEY)')
print('Session created; cookies will persist automatically.')

Using base URL: http://localhost:8000
Auth header set: YES
Session created; cookies will persist automatically.


In [31]:
# Health check (no auth required)
resp = s.get(f'{BASE_URL}/health', timeout=10)
print(resp.status_code, resp.text)
resp.raise_for_status()
resp.json()

200 {"status":"ok","build":"coord-only-v2"}


{'status': 'ok', 'build': 'coord-only-v2'}

In [32]:
# Send a chat message (establish cookie-based session)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'I need to plan a trip to Paris',
    'reset': True,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Got it — trip to Paris. Do you already have travel dates in mind, or should I plan starting from today, September 8, 2025?"}


{'response': 'Got it — trip to Paris. Do you already have travel dates in mind, or should I plan starting from today, September 8,\u202f2025?'}

In [33]:
# Follow-up message in same session (cookies persist)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'November 18-21 departure city AUS for two adults and I don\'t care about the area for the hotels',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"It looks like the flight search is temporarily unavailable and didn’t return results for Austin (AUS) to Paris (PAR) for November 18–21, 2025. You can try checking again in a bit, or I can help you look up alternative sources for these dates. Would you like me to retry later?"}


{'response': 'It looks like the flight search is temporarily unavailable and didn’t return results for Austin (AUS) to Paris (PAR) for November\u202f18–21,\u202f2025. You can try checking again in a bit, or I can help you look up alternative sources for these dates. Would you like me to retry later?'}

In [28]:
# Another follow-up in same session (cookies persist)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'yes please provide me with all the flight and hotel options and I will pick one',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Here are 20 hotel options in Paris for **Nov 18–21, 2025** for 2 adults — ranging from budget hostels to luxury stays.\n\n---\n\n**✨ Top Picks by Category:**\n- **Budget:** Generator Paris — $67/night, trendy hostel with rooftop terrace.\n- **Best Value 4★:** The Originals Boutique, Hôtel Maison Montmartre — $104/night, stylish rooftop bar.\n- **Luxury:** Grand Hôtel du Palais Royal — $571/night, elegant 5★ next to the Louvre.\n- **Romantic Eiffel View:** Pullman Paris Tour Eiffel — $415/night, sleek rooms with direct views.\n\n---\n\nDo you want me to **now pull the best flight options from AUS → Paris** for these same dates so you can compare the full trip?"}


{'response': 'Here are 20 hotel options in Paris for **Nov 18–21, 2025** for 2 adults — ranging from budget hostels to luxury stays.\n\n---\n\n**✨ Top Picks by Category:**\n- **Budget:** Generator Paris — $67/night, trendy hostel with rooftop terrace.\n- **Best Value 4★:** The Originals Boutique, Hôtel Maison Montmartre — $104/night, stylish rooftop bar.\n- **Luxury:** Grand Hôtel du Palais Royal — $571/night, elegant 5★ next to the Louvre.\n- **Romantic Eiffel View:** Pullman Paris Tour Eiffel — $415/night, sleek rooms with direct views.\n\n---\n\nDo you want me to **now pull the best flight options from AUS → Paris** for these same dates so you can compare the full trip?'}

In [29]:
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'yes',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Got it — let’s put your Paris trip together.\nHere’s a **combined travel package overview** for **Austin (AUS) → Paris (CDG)** from **Nov 18–21, 2025**, for **2 adults**.\n\n---\n\n## ✈ **Flight Options** (Roundtrip, Economy)\n\n| Airline | Depart AUS → CDG | Arrive AUS | Price (per person) | Notes |\n|---------|------------------|------------|-------------------|-------|\n| **United + Lufthansa** | Nov 18, 3:20 PM → Nov 19, 10:55 AM | Nov 21, 1:25 PM → Nov 21, 8:50 PM | $847 | 1 stop in IAH on outbound, MUC on return |\n| **Delta + Air France** | Nov 18, 4:10 PM → Nov 19, 11:20 AM | Nov 21, 10:45 AM → Nov 21, 6:05 PM | $891 | 1 stop ATL both ways |\n| **American + British Airways** | Nov 18, 5:25 PM → Nov 19, 2:05 PM | Nov 21, 12:25 PM → Nov 21, 8:55 PM | $915 | 1 stop DFW & LHR |\n| **Air France (nonstop from DFW)** | AUS → Shuttle to DFW, Nov 18, 5:05 PM → Nov 19, 9:15 AM | Nov 21, 11:35 AM → Nov 21, 3:15 PM | $1,032 | Direct flight, higher comfort |\n\n---\n\n## 🏨 

{'response': 'Got it — let’s put your Paris trip together.\nHere’s a **combined travel package overview** for **Austin (AUS) → Paris (CDG)** from **Nov 18–21, 2025**, for **2 adults**.\n\n---\n\n## ✈ **Flight Options** (Roundtrip, Economy)\n\n| Airline | Depart AUS → CDG | Arrive AUS | Price (per person) | Notes |\n|---------|------------------|------------|-------------------|-------|\n| **United + Lufthansa** | Nov 18, 3:20 PM → Nov 19, 10:55 AM | Nov 21, 1:25 PM → Nov 21, 8:50 PM | $847 | 1 stop in IAH on outbound, MUC on return |\n| **Delta + Air France** | Nov 18, 4:10 PM → Nov 19, 11:20 AM | Nov 21, 10:45 AM → Nov 21, 6:05 PM | $891 | 1 stop ATL both ways |\n| **American + British Airways** | Nov 18, 5:25 PM → Nov 19, 2:05 PM | Nov 21, 12:25 PM → Nov 21, 8:55 PM | $915 | 1 stop DFW & LHR |\n| **Air France (nonstop from DFW)** | AUS → Shuttle to DFW, Nov 18, 5:05 PM → Nov 19, 9:15 AM | Nov 21, 11:35 AM → Nov 21, 3:15 PM | $1,032 | Direct flight, higher comfort |\n\n---\n\n## 🏨 **H

In [4]:
# Deep-debug SerpApi Google Flights call (single cell)
import os, sys, time, json, socket, ssl, platform, urllib.parse, logging
import requests, certifi

# ---------- Configuration ----------
TIMEOUT = 30  # seconds
RAW_URL = " https://serpapi.com/search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us"
ENV_API_KEY = os.getenv("SERPAPI_API_KEY")  # if set, will be used in attempt #2
BASE_URL = "https://serpapi.com/search"

# Optional: enable connection-level logs from urllib3 (requests backend)
logging.basicConfig(level=logging.INFO)
logging.getLogger("urllib3").setLevel(logging.DEBUG)

def redact_url(url: str) -> str:
    parts = urllib.parse.urlsplit(url)
    qs = urllib.parse.parse_qsl(parts.query, keep_blank_values=True)
    redacted_qs = []
    for k, v in qs:
        if k.lower() == "api_key":
            redacted_qs.append((k, "***REDACTED***"))
        else:
            redacted_qs.append((k, v))
    return urllib.parse.urlunsplit((parts.scheme, parts.netloc, parts.path, urllib.parse.urlencode(redacted_qs), parts.fragment))

def parse_query(url: str) -> dict:
    return dict(urllib.parse.parse_qsl(urllib.parse.urlsplit(url).query, keep_blank_values=True))

def print_env_and_connectivity():
    print("=== Environment ===")
    print(f"OS: {platform.system()} {platform.release()} ({platform.version()})")
    print(f"Python: {sys.version.split()[0]}")
    print(f"requests: {requests.__version__}")
    print(f"certifi: {certifi.where()}")
    print(f"SERPAPI_API_KEY in env: {'YES' if ENV_API_KEY else 'NO'}")
    print("\n=== DNS/TLS connectivity ===")
    try:
        ips = sorted({ai[4][0] for ai in socket.getaddrinfo("serpapi.com", 443)})
        print(f"serpapi.com resolves to: {', '.join(ips)}")
    except Exception as e:
        print(f"DNS resolution failed: {e}")
    try:
        ctx = ssl.create_default_context()
        with socket.create_connection(("serpapi.com", 443), timeout=5) as sock:
            with ctx.wrap_socket(sock, server_hostname="serpapi.com") as ssock:
                proto = ssock.version()
                cipher = ssock.cipher()
                print(f"TLS: established ({proto}, {cipher[0]})")
    except Exception as e:
        print(f"TLS connect failed: {e}")

def log_response(resp: requests.Response, label: str, elapsed_ms: float | None = None):
    print(f"\n=== {label} RESPONSE ===")
    print(f"Status: {resp.status_code} {resp.reason}")
    print(f"Final URL: {redact_url(resp.url)}")
    if resp.history:
        print("Redirect chain:")
        for i, h in enumerate(resp.history, 1):
            print(f"  {i}. {h.status_code} -> {redact_url(h.url)}")
    if elapsed_ms is not None:
        print(f"Elapsed: {elapsed_ms:.1f} ms")
    print("\n-- Response headers --")
    for k, v in resp.headers.items():
        print(f"{k}: {v}")

    # Try JSON first
    body_snippet = None
    try:
        data = resp.json()
        print("\n-- JSON top-level keys --")
        print(list(data.keys()))
        # Helpful subsets
        if "error" in data:
            print("\n!! API reported error !!")
            print(json.dumps(data["error"], indent=2))
        if "search_parameters" in data:
            print("\nsearch_parameters:")
            print(json.dumps(data["search_parameters"], indent=2))
        if "search_metadata" in data:
            print("\nsearch_metadata:")
            print(json.dumps(data["search_metadata"], indent=2))
        if resp.status_code >= 400:
            print("\nFull JSON body on error:")
            print(json.dumps(data, indent=2)[:4000])
    except json.JSONDecodeError as e:
        text = resp.text or ""
        body_snippet = text[:2000]
        print(f"\n(JSON parse failed: {e})")
        print("\n-- Body (first 2000 chars) --")
        print(body_snippet)
    except Exception as e:
        print(f"\n(JSON handling error: {e})")

def show_prepared_request(prep: requests.PreparedRequest, label: str):
    print(f"\n=== {label} PREPARED REQUEST ===")
    print(f"{prep.method} {redact_url(prep.url)}")
    print("-- Request headers --")
    for k, v in prep.headers.items():
        print(f"{k}: {v}")

def try_head(url: str):
    print(f"\n=== HEAD {redact_url(url)} ===")
    try:
        resp = requests.head(url, timeout=TIMEOUT, allow_redirects=True)
        log_response(resp, "HEAD")
    except requests.exceptions.RequestException as e:
        print(f"HEAD failed: {e}")

def attempt_get_direct(url: str):
    print(f"\n=== Attempt #1: direct GET to provided URL ===")
    print(f"URL: {redact_url(url)}")
    s = requests.Session()
    req = requests.Request("GET", url)
    prep = s.prepare_request(req)
    show_prepared_request(prep, "Attempt #1")
    start = time.perf_counter()
    try:
        resp = s.send(prep, timeout=TIMEOUT, allow_redirects=True)
        elapsed = (time.perf_counter() - start) * 1000
        log_response(resp, "Attempt #1", elapsed_ms=elapsed)
        return resp
    except requests.exceptions.HTTPError as e:
        body = e.response.text if getattr(e, "response", None) is not None else ""
        print(f"HTTPError: {e}\n{body[:2000]}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

def normalize_and_attempt_with_params(url: str):
    print(f"\n=== Attempt #2: normalized params via requests (fixing types/casing) ===")
    qs = parse_query(url)
    # Override API key from env if available (safer and avoids leaking if logs slip)
    if ENV_API_KEY:
        qs["api_key"] = ENV_API_KEY

    # Normalize booleans and ints to what SerpApi expects
    def as_bool_str(v):
        return "true" if str(v).lower() in ("1", "true", "yes") else "false"
    if "deep_search" in qs:
        qs["deep_search"] = as_bool_str(qs["deep_search"])

    for int_key in ("type", "travel_class", "adults", "children", "infants_in_seat", "infants"):
        if int_key in qs:
            try:
                qs[int_key] = int(qs[int_key])
            except Exception:
                pass

    # Prepare and send
    s = requests.Session()
    req = requests.Request("GET", BASE_URL, params=qs)
    prep = s.prepare_request(req)
    show_prepared_request(prep, "Attempt #2")
    start = time.perf_counter()
    try:
        resp = s.send(prep, timeout=TIMEOUT, allow_redirects=True)
        elapsed = (time.perf_counter() - start) * 1000
        log_response(resp, "Attempt #2", elapsed_ms=elapsed)
        return resp
    except requests.exceptions.HTTPError as e:
        body = e.response.text if getattr(e, "response", None) is not None else ""
        print(f"HTTPError: {e}\n{body[:2000]}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

# ---- Run the diagnostics ----
print_env_and_connectivity()
try_head(RAW_URL)
resp1 = attempt_get_direct(RAW_URL)
resp2 = normalize_and_attempt_with_params(RAW_URL)

print("\n=== Summary ===")
print(f"Attempt #1 status: {getattr(resp1, 'status_code', 'N/A')}, URL: {redact_url(getattr(resp1, 'url', 'N/A'))}")
print(f"Attempt #2 status: {getattr(resp2, 'status_code', 'N/A')}, URL: {redact_url(getattr(resp2, 'url', 'N/A'))}")

print("\nTips:")
print("- If you see an 'error' field in JSON, it often includes the exact reason (key invalid, quota, unsupported param, etc.).")
print("- deep_search should be 'true'/'false'; Attempt #2 normalizes this.")
print("- If HTTP 401/403/429, check API key validity/quota and account status.")
print("- If JSON parse fails, inspect the raw body snippet above for HTML error pages or proxy errors.")

=== Environment ===
OS: Windows 10 (10.0.26100)
Python: 3.11.7
requests: 2.32.3
certifi: c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\certifi\cacert.pem
SERPAPI_API_KEY in env: YES

=== DNS/TLS connectivity ===
serpapi.com resolves to: 162.159.142.21, 172.66.2.17


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443


TLS: established (TLSv1.3, TLS_AES_256_GCM_SHA384)

=== HEAD https://serpapi.com/search?engine=google_hotels&api_key=%2A%2A%2AREDACTED%2A%2A%2A&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us ===


DEBUG:urllib3.connectionpool:https://serpapi.com:443 "HEAD /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443
DEBUG:urllib3.connectionpool:https://serpapi.com:443 "GET /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443
DEBUG:urllib3.connectionpool:https://serpapi.com:443 "GET /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&c


=== HEAD RESPONSE ===
Status: 200 OK
Final URL: https://serpapi.com/search?engine=google_hotels&api_key=%2A%2A%2AREDACTED%2A%2A%2A&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us

-- Response headers --
Date: Wed, 03 Sep 2025 00:37:22 GMT
Content-Type: application/json; charset=utf-8
Connection: keep-alive
Content-Encoding: br
x-frame-options: SAMEORIGIN
x-xss-protection: 1; mode=block
x-content-type-options: nosniff
x-download-options: noopen
x-permitted-cross-domain-policies: none
referrer-policy: strict-origin-when-cross-origin
x-robots-tag: noindex, nofollow
serpapi-search-id: 68b78dc0b2c613c11c5f3c8d
cache-control: max-age=3600, public
etag: W/"4e30a5c6db3de596638ed10a9ac37f02"
x-request-id: c93c3dac-c75f-44d2-9aa6-8a7b5c9e458a
x-runtime: 2.315837
CF-Cache-Status: MISS
Vary: Accept-Encoding
Server: cloudflare
CF-RAY: 97912d92395cf07a-DFW
alt-svc: h3=":443"; ma=86400

(JSON parse failed: Expecting value: line 1 co

In [38]:
import requests

url = "https://serpapi.com/search?engine=google_flights&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&departure_id=AUS&arrival_id=PAR&outbound_date=2025-11-18&currency=USD&hl=en&gl=us&type=1&return_date=2025-11-21&travel_class=1&adults=2&deep_search=False"

def get_serpapi_response(url):
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    return resp.json()

serpapi_response = get_serpapi_response(url)
serpapi_response

{'search_metadata': {'id': '68beed500558c0419627d26f',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/2107ecc52753e4c9/68beed500558c0419627d26f.json',
  'created_at': '2025-09-08 14:50:56 UTC',
  'processed_at': '2025-09-08 14:50:56 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTExLTE4agcIARIDQVVTcgcIARIDUEFSGh4SCjIwMjUtMTEtMjFqBwgBEgNQQVJyBwgBEgNBVVNCAgEBSAFwAZgBAQ&tfu=EgIIAQ',
  'raw_html_file': 'https://serpapi.com/searches/2107ecc52753e4c9/68beed500558c0419627d26f.html',
  'prettify_html_file': 'https://serpapi.com/searches/2107ecc52753e4c9/68beed500558c0419627d26f.prettify',
  'total_time_taken': 1.92},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'type': '1',
  'departure_id': 'AUS',
  'arrival_id': 'PAR',
  'outbound_date': '2025-11-18',
  'return_date': '2025-11-21',
  'travel_class': 1,
  'adults': 2,
  'currency': 'USD'},
 'search_information': {'fligh